In [29]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv("prosperLoanData.csv") # the full dataset

### Dropped column names

In [3]:
dropped = ['ListingNumber', 'LoanNumber', 'GroupKey', 'LoanKey', 'MemberKey', 'ListingKey']

In [4]:
for col in data.columns :
    if (data[col].isnull().sum() > 0) and col not in dropped:
        sum_null = data[col].isnull().sum()
        percent_null = (sum_null/data.shape[0]) * 100
        if percent_null > 70 :
            dropped.append(col)

## Data Preprocessing

### Imputing `CreditGrade` & `ProsperRating (Alpha)` 

In [5]:
data['CreditGrade'].fillna(data['ProsperRating (Alpha)'],inplace=True)

In [6]:
dropped.extend(['ProsperRating (Alpha)','ProsperRating (numeric)'])
dropped.remove('CreditGrade')

In [7]:
data.drop(dropped,axis=1,inplace=True)

### Filling missing values

In [8]:
data.isnull().sum().sort_values(ascending=False)

ClosedDate                 58848
EstimatedReturn            29084
ProsperScore               29084
EstimatedEffectiveYield    29084
EstimatedLoss              29084
                           ...  
LenderYield                    0
BorrowerRate                   0
LoanStatus                     0
Term                           0
Investors                      0
Length: 64, dtype: int64

In [9]:
for col in data.select_dtypes('float64').columns:
    if data[col].isnull().sum() > 0: 
        data[col].fillna(data[col].median(),inplace=True)
data.select_dtypes('float64').isnull().sum().sum()

0

In [10]:
for col in data.select_dtypes('int64').columns: 
    if data[col].nunique() > 100:
        data[col].fillna(data[col].median(), inplace=True)
    else:
        data[col].fillna(data[col].mode(), inplace=True)
data.select_dtypes('int64').isnull().sum().sum()

0

In [11]:
for col in data.columns :
    if data[col].isnull().sum() > 0 :
        data[col].fillna(data[col].mode()[0],inplace=True)
data.isnull().sum().sum()

0

### Imputing Dates

In [12]:
date =['FirstRecordedCreditLine','DateCreditPulled','ListingCreationDate','ClosedDate','LoanOriginationDate']
# convert date columns from object to datetime 
for i in date:
    #to get ride of time convert data to datetime
    data[i]=pd.to_datetime(data[i])
    data[i]=data[i].dt.date  # datatype=object
    # split each coloumn to year month day then delet it 
    data[f'{i}_year']= pd.DatetimeIndex(data[f'{i}']).year
    data[f'{i}_month']= pd.DatetimeIndex(data[f'{i}']).month
    data[f'{i}_day']= pd.DatetimeIndex(data[f'{i}']).day
    del(data[f'{i}'])

### Detecting Outliers

In [13]:
for col in data.columns:
        if (((data[col].dtype)=='float64') | ((data[col].dtype)=='int64')):
            percentiles = data[col].quantile([0.25,0.75]).values
            iqr = percentiles[1] - percentiles[0]
            lower_bound = percentiles[0] -(1.5 * iqr) 
            upper_bound = percentiles[1] +(1.5 * iqr)
            data[col].loc[data[col] < lower_bound] = lower_bound
            data[col].loc[data[col] > upper_bound] = upper_bound
        else:
            data[col]=data[col]

c:\users\alaa_\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### Imputing Target Value

In [14]:
def target_handling(row): 
    if row == 'Completed':
        return 1
    else:
        return 0

data['Status'] = data['LoanStatus'].apply(target_handling)
data.drop('LoanStatus',axis=1,inplace=True)

### Encoding 

In [15]:
obj = data.select_dtypes('object')
obj.nunique()

CreditGrade                8
BorrowerState             51
Occupation                67
EmploymentStatus           8
IncomeRange                8
LoanOriginationQuarter    33
dtype: int64

In [16]:
### one hot encoding
dummies = pd.get_dummies(data[['CreditGrade', 'EmploymentStatus', 'IncomeRange','LoanOriginationQuarter']], drop_first=True)

In [17]:
## label enc
le = LabelEncoder()
data[['BorrowerState','Occupation']] = data[['BorrowerState','Occupation']].apply(le.fit_transform)

In [18]:
data = pd.concat([data.drop(obj,axis=1),dummies],axis=1)

In [19]:
data.select_dtypes('object')

""
0
1
2
3
4
...
113932
113933
113934
113935


In [ ]:
data

,Term,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperScore,ListingCategory (numeric),EmploymentStatusDuration,...,LoanOriginationQuarter_Q3 2013,LoanOriginationQuarter_Q4 2005,LoanOriginationQuarter_Q4 2006,LoanOriginationQuarter_Q4 2007,LoanOriginationQuarter_Q4 2008,LoanOriginationQuarter_Q4 2009,LoanOriginationQuarter_Q4 2010,LoanOriginationQuarter_Q4 2011,LoanOriginationQuarter_Q4 2012,LoanOriginationQuarter_Q4 2013
0,36,0.16516,0.1580,0.1380,0.16150,0.0724,0.09170,6.0,0,2.0,...,0,0,0,0,0,0,0,0,0,0
1,36,0.12016,0.0920,0.0820,0.07960,0.0249,0.05470,7.0,2,44.0,...,0,0,0,0,0,0,0,0,0,0
2,36,0.28269,0.2750,0.2400,0.16150,0.0724,0.09170,6.0,0,67.0,...,0,0,0,0,0,0,0,0,0,0
3,36,0.12528,0.0974,0.0874,0.08490,0.0249,0.06000,9.0,6,113.0,...,0,0,0,0,0,0,0,0,1,0
4,36,0.24614,0.2085,0.1985,0.18316,0.0925,0.09066,4.0,2,44.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113932,36,0.22354,0.1864,0.1764,0.16490,0.0699,0.09500,5.0,1,246.0,...,0,0,0,0,0,0,0,0,0,0
113933,36,0.13220,0.1110,0.1010,0.10070,0.0200,0.08070,8.0,6,21.0,...,0,0,0,0,0,0,0,1,0,0
113934,36,0.23984,0.2150,0.2050,0.18828,0.1025,0.08578,3.0,1,84.0,...,0,0,0,0,0,0,0,0,0,1
113935,36,0.28408,0.2605,0.2505,0.24450,0.0850,0.15142,5.0,2,94.0,...,0,0,0,0,0,0,0,1,0,0


### Feature Selection

In [21]:
corrlation = data.corrwith(data['Status']).sort_values(ascending = False)
sel_col = corrlation[(corrlation > 0.2) | (corrlation < -0.2)]
sel_col.drop('Status',inplace=True)

In [22]:
sel_col

LP_CustomerPrincipalPayments      0.598982
LoanMonthsSinceOrigination        0.510850
LP_CustomerPayments               0.495638
ClosedDate_day                    0.490288
ClosedDate_month                  0.402958
EmploymentStatus_Full-time        0.379045
Investors                         0.209813
MonthlyLoanPayment               -0.211909
LoanOriginationQuarter_Q1 2014   -0.240852
LoanOriginalAmount               -0.246836
LoanOriginationQuarter_Q4 2013   -0.248185
ClosedDate_year                  -0.465317
DateCreditPulled_year            -0.507319
LoanOriginationDate_year         -0.508132
ListingCreationDate_year         -0.508645
dtype: float64

## Modeling

In [23]:
X = data[sel_col.index]
y= data['Status']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [25]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
pred = clf.predict(X_test)
accuracy_score(y_test,pred)

c:\users\alaa_\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9401595744680851

In [26]:
parms = {
    'C':[0.01, 0.1, 1],
    'solver': ['lbfgs', 'liblinear', 'sag'],     
}
LR = LogisticRegression() 
clf = GridSearchCV(LR, parms)
clf.fit(X_train,y_train)

c:\users\alaa_\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\alaa_\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

GridSearchCV(estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1],
                         'solver': ['lbfgs', 'liblinear', 'sag']})

In [27]:
clf.best_params_

{'C': 1, 'solver': 'sag'}

In [31]:
clf = LogisticRegression(random_state=0,C=.01,solver='sag').fit(X_train, y_train)
pred = clf.predict(X_test)
accuracy_score(y_test,pred)

c:\users\alaa_\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.9427393617021277

### Model Evaluation

In [30]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.96     24959
           1       0.91      0.93      0.92     12641

    accuracy                           0.94     37600
   macro avg       0.93      0.94      0.94     37600
weighted avg       0.94      0.94      0.94     37600

